In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

In [2]:
nummerch=15000
numusers=15000
fracfraud=0.1
numtransactions=10000
numfraud=fracfraud*numtransactions
numtransactions=numtransactions-numfraud
totnumtrans=numtransactions+numfraud
totnumtrans,numtransactions,numfraud

(10000.0, 9000.0, 1000.0)

In [3]:
dfcc=pd.read_csv('creditcard.csv')
dfhc=pd.read_csv('Price.csv')

In [4]:
amounts=dfcc.Amount.unique()

In [5]:
timerange=24*14
def timefn(r=random.random()):
    #r=random.random()
    return timerange*(math.sin(r*2*math.pi*(timerange/24))+1)
timefn()

def agefn():
    r=random.randrange(18,80)
    return r


def genderfn():
    r=random.choices(['m','f'],weights=[0.485,0.515])
    return r[0]


numbanks=15
banks=["Bank{:d}".format(b) for b in range(numbanks) ]
bw   =[(numbanks-b)*(numbanks-b) for b in range(numbanks) ]
bw=bw/np.sum(bw)
def bankfn():
    r=random.choices(banks,weights=bw)
    return r[0]


cards=['Visa',"Visa_Debit","Master","Master_Debit","AmEx"]
cw=[(len(cards)-c)*(len(cards)-c)for c in range(len(cards))]
cw=cw/np.sum(cw)
def cardfn():
    r=random.choices(cards,weights=cw)
    return r[0]

def cardnumfn():
    s=""
    for i in range(4):
        s=s+"{:04d} ".format(random.randint(1,9999))
    return s.strip()

def cityfn():
    c=random.choice(dfhc.city)
    return(c)

entry=['magnetic','chip']
ew=[0.3,0.7]
def entryfn():
    c=random.choices(entry,weights=ew)
    return c[0]

trans=['ATM','POS','INTERNET']
tw=[0.25,0.35,0.4]
def transfn():
    c=random.choices(trans,weights=tw)
    return c[0]

def newcard():
    return {"type":cardfn(),"num":cardnumfn(),"bank":bankfn()}

def chosenumcardsfn():
    numcards=[1,2,3,4,5,6]
    ncw=[32,64,32,16,8,4]
    ncw=ncw/np.sum(ncw)
    return random.choices(numcards,ncw)[0]

cmap={}
cmap['Visa']=4
cmap["Visa_Debit"]=2
cmap["Master"]=3
cmap["Master_Debit"]=1
cmap["AmEx"]=3
def cardweightfn(card):
    return cmap[card['type']]

In [6]:
def userfn():
    card=[]
    prob=[]
    for i in range(chosenumcardsfn()):
        card.append(newcard())
        prob.append(random.random())
    if len(card)<1:
        card.append(newcard())
        prob.append(random.random())
    prob=prob/np.sum(prob)
    tp=0.2*random.random()
    idnum="U{:09d}".format(random.randint(0,1e9))
    return {"id":idnum,"gender":genderfn(),"age":agefn(),"rescity":cityfn(),"cards":card,"carduseprob":prob,"travelprob":tp}

#timefn(),agefn(),agefn(),genderfn(),bankfn(),cardfn(),cardnumfn(),cityfn(),entryfn(),transfn()
userfn()

{'id': 'U344775541',
 'gender': 'm',
 'age': 70,
 'rescity': 'Yonkers',
 'cards': [{'type': 'Visa_Debit',
   'num': '6340 9665 9326 6663',
   'bank': 'Bank4'}],
 'carduseprob': array([1.]),
 'travelprob': 0.14164126096410035}

In [7]:
mcc=pd.read_csv("mcc.csv",sep="\t")
def merchfn():
    idnum="M{:06d}".format(random.randint(0,1e6))
    return {"type":random.choice(mcc.MCC.unique()),"id":idnum,"city":cityfn(),"trans":transfn()}
merchfn()

{'type': 5571, 'id': 'M621428', 'city': 'New Haven', 'trans': 'ATM'}

In [8]:
u=userfn()
u['age']

53

In [9]:

users=[]
for i in range(numusers):
    users.append(userfn())
dfu=pd.DataFrame(users)
dfu.shape

(15000, 7)

In [10]:

merchs=[]
for i in range(nummerch):
    merchs.append(merchfn())
dfm=pd.DataFrame(merchs)
dfm.shape

(15000, 4)

In [11]:
def genTransaction(user,time):

    cardchoice=random.randint(0,len(user["cards"])-1)
    card=user["cards"][cardchoice]

    
    tra=transfn()
    ent=entryfn()
    rtravel=random.random()
    if (tra=='POS' or tra=='ATM') and rtravel>user["travelprob"]:
        merch=dfm[(dfm['trans']==tra) & (dfm['city']==user["rescity"])].sample(1) ####and put in && dfm['city']==user["rescity"]
    elif (tra=='POS' or tra=='ATM') and rtravel< user["travelprob"]:
        merch=dfm[dfm['trans']==tra].sample(1)
    else:
        merch=dfm[dfm['trans']=="INTERNET"].sample(1)
        ent="INTERNET"
    
    mc=merch["city"].values[0]
    rc=user["rescity"]
    mt=merch["type"].values[0]
    
    datasample=dfcc[dfcc["Class"]==0].sample(1)
    amount=datasample['Amount'].values[0]
    v1=datasample['V1'].values[0]
    v2=datasample['V2'].values[0]
    v3=datasample['V3'].values[0]
    v4=datasample['V4'].values[0]
    v5=datasample['V5'].values[0]
    v6=datasample['V6'].values[0]
    v7=datasample['V7'].values[0]
    v8=datasample['V8'].values[0]
    v9=datasample['V9'].values[0]
    v10=datasample['V10'].values[0]
    
    tr={"time":time,
        "user_id":user['id'],
        'Age':user['age'],
        'gender':user['gender'],
        'bank':card['bank'],
        'cardType':card["type"],
        'card_number':card["num"],
        'entry_type':ent,
        'city':rc,
        'city_trans':mc,
        'merchant_type':mt,
        'transaction':tra,
        'trans_number':"T{:09d}".format(random.randint(0,1e9)),
        "Amount":amount,
        "fraud":0,
        "v1":v1,
        "v2":v2,
        "v3":v3,
        "v4":v4,
        "v5":v5,
        "v6":v6,
        "v7":v7,
        "v8":v8,
        "v9":v9,
        "v10":v10,
       }
    #print(tr)
    return tr
    

In [12]:
def genFraud(user,time):

    cardindexes=[i for i in range(len(user["cards"]))]
    cardweights=[cardweightfn(user["cards"][i]) for i in cardindexes]
    cardweights=cardweights/np.sum(cardweights)
    
    cardchoice=random.choices(cardindexes,cardweights)

    card=user["cards"][cardchoice[0]]

    trans=['ATM','POS','INTERNET']
    tw=[0.3,0.2,0.5]
    tw=tw/np.sum(tw)
    tra=random.choices(trans,weights=tw)[0]

    entry=['magnetic','chip']
    ew=[0.7,0.3]
    ew=ew/np.sum(ew)
    ent=random.choices(entry,weights=ew)[0]
    
    if tra=='POS' or tra=='ATM':
        merch=dfm[dfm['trans']==tra].sample(1)
    else:
        merch=dfm[dfm['trans']=="INTERNET"].sample(1)
        ent="internet"
    
    mc=merch["city"].values[0]
    rc=user["rescity"]
    mt=merch["type"].values[0]
    
    datasample=dfcc[dfcc["Class"]==1].sample(1)
    amount=datasample['Amount'].values[0]
    v1=datasample['V1'].values[0]
    v2=datasample['V2'].values[0]
    v3=datasample['V3'].values[0]
    v4=datasample['V4'].values[0]
    v5=datasample['V5'].values[0]
    v6=datasample['V6'].values[0]
    v7=datasample['V7'].values[0]
    v8=datasample['V8'].values[0]
    v9=datasample['V9'].values[0]
    v10=datasample['V10'].values[0]
    
    tr={"time":time,
        "user_id":user['id'],
        'Age':user['age'],
        'gender':user['gender'],
        'bank':card['bank'],
        'cardType':card["type"],
        'card_number':card["num"],
        'entry_type':ent,
        'city':rc,
        'city_trans':mc,
        'merchant_type':mt,
        'transaction':tra,
        'trans_number':"T{:09d}".format(random.randint(0,1e9)),
        "Amount":amount*10,
        "fraud":1,
        "v1":v1,
        "v2":v2,
        "v3":v3,
        "v4":v4,
        "v5":v5,
        "v6":v6,
        "v7":v7,
        "v8":v8,
        "v9":v9,
        "v10":v10,
       }
    #print(tr)
    return tr
    

In [13]:
transactions=[]
#make bad transactions
for i in range(int(numfraud)):
    v=random.choice(range(dfu.shape[0]))
    transactions.append(genFraud(dfu.iloc[v],timefn()))
    if i%1000==0:
        print(i)

#make good transactions
for i in range(int(numtransactions)):
    v=random.choice(range(dfu.shape[0]))
    transactions.append(genTransaction(dfu.iloc[v],timefn()))
    if i%1000==0:
        print(i)
        
ta=pd.DataFrame(transactions)
ta

0
0
1000
2000
3000
4000
5000
6000
7000
8000


,Age,Amount,bank,cardType,card_number,city,city_trans,entry_type,fraud,gender,...,v1,v10,v2,v3,v4,v5,v6,v7,v8,v9
0,45,170.60,Bank8,Visa_Debit,0439 0745 0426 0730,Camden,New York City,chip,1,f,...,-1.550273,-3.514133,1.088689,-2.393388,1.008733,-1.087562,-1.104602,-2.670503,0.147655,-0.978626
1,70,4441.70,Bank12,Visa,5941 2403 6224 3030,Watertown,Jersey City,internet,1,f,...,-2.335655,-5.711505,2.225380,-3.379450,2.178538,-3.568264,0.316814,-1.734948,1.449139,-1.980033
2,23,10.00,Bank0,Visa_Debit,8476 9335 1880 7880,Newton,Danbury,chip,1,f,...,-3.843009,-4.565252,3.375110,-5.492893,6.136378,2.797195,-2.646162,-1.668931,-2.617552,-3.945843
3,32,11.80,Bank6,Visa_Debit,9484 5109 6238 5097,Hartford,Bronx,magnetic,1,m,...,1.296231,-0.045088,0.417447,0.193963,0.901644,0.130531,-0.371634,0.158126,-0.202669,-0.079512
4,74,37.90,Bank2,Master,3754 6260 1608 8607,Erie,Hartford,internet,1,m,...,-3.600544,-11.298156,4.519047,-6.340884,6.214767,-5.829558,-2.478095,-9.938412,2.830086,-5.659162
5,21,224.70,Bank0,Visa,0602 4486 8531 0153,Providence,Syracuse,chip,1,m,...,-4.666500,4.031435,-3.952320,0.206094,5.153525,5.229469,0.939040,-0.635033,-0.704506,-0.234786
6,25,15.20,Bank1,Visa_Debit,5464 3312 4795 2269,Paterson,Newark,internet,1,m,...,1.081234,0.644550,0.416414,0.862919,2.520863,-0.005021,0.563341,-0.123372,0.223122,-0.673598
7,31,180.00,Bank6,Visa,8851 9895 6394 4008,Camden,Newton,internet,1,m,...,-1.108478,-4.095649,3.448953,-6.216972,3.021052,-0.529901,-2.551375,-2.001743,1.092432,-0.836098
8,78,1302.10,Bank2,Visa_Debit,7494 6468 8126 2565,New York City,Rochester,magnetic,1,f,...,0.523820,-2.649406,1.531708,-4.176390,3.584615,-1.023954,-0.502471,-1.891966,0.878417,-1.541942
9,21,1861.30,Bank6,Visa_Debit,1616 8157 5881 1644,West Hartford,New Haven,internet,1,m,...,-1.000611,-4.458008,3.346850,-5.534491,6.835802,-0.299803,0.095951,-2.440419,1.286301,-2.766438


In [14]:
#["user_id",'Age','gender','bank','cardType','card_number','city','city_trans','merchant_type','entry_type','trans_type','trans_number']

In [15]:
colsout=['Age', 'Amount', 'bank', 'cardType', 'entry_type', 'gender',
         'transaction', 'v1', 'v2', 'v3', 'v4','fraud']

In [16]:
ta[colsout].sample(frac=1).to_csv("out.csv",index=False)

In [17]:
ta[(ta['entry_type']!="INTERNET")&(ta['city']!=ta["city_trans"])].shape[0]/ta.shape[0]

0.1458

In [18]:
ta


,Age,Amount,bank,cardType,card_number,city,city_trans,entry_type,fraud,gender,...,v1,v10,v2,v3,v4,v5,v6,v7,v8,v9
0,45,170.60,Bank8,Visa_Debit,0439 0745 0426 0730,Camden,New York City,chip,1,f,...,-1.550273,-3.514133,1.088689,-2.393388,1.008733,-1.087562,-1.104602,-2.670503,0.147655,-0.978626
1,70,4441.70,Bank12,Visa,5941 2403 6224 3030,Watertown,Jersey City,internet,1,f,...,-2.335655,-5.711505,2.225380,-3.379450,2.178538,-3.568264,0.316814,-1.734948,1.449139,-1.980033
2,23,10.00,Bank0,Visa_Debit,8476 9335 1880 7880,Newton,Danbury,chip,1,f,...,-3.843009,-4.565252,3.375110,-5.492893,6.136378,2.797195,-2.646162,-1.668931,-2.617552,-3.945843
3,32,11.80,Bank6,Visa_Debit,9484 5109 6238 5097,Hartford,Bronx,magnetic,1,m,...,1.296231,-0.045088,0.417447,0.193963,0.901644,0.130531,-0.371634,0.158126,-0.202669,-0.079512
4,74,37.90,Bank2,Master,3754 6260 1608 8607,Erie,Hartford,internet,1,m,...,-3.600544,-11.298156,4.519047,-6.340884,6.214767,-5.829558,-2.478095,-9.938412,2.830086,-5.659162
5,21,224.70,Bank0,Visa,0602 4486 8531 0153,Providence,Syracuse,chip,1,m,...,-4.666500,4.031435,-3.952320,0.206094,5.153525,5.229469,0.939040,-0.635033,-0.704506,-0.234786
6,25,15.20,Bank1,Visa_Debit,5464 3312 4795 2269,Paterson,Newark,internet,1,m,...,1.081234,0.644550,0.416414,0.862919,2.520863,-0.005021,0.563341,-0.123372,0.223122,-0.673598
7,31,180.00,Bank6,Visa,8851 9895 6394 4008,Camden,Newton,internet,1,m,...,-1.108478,-4.095649,3.448953,-6.216972,3.021052,-0.529901,-2.551375,-2.001743,1.092432,-0.836098
8,78,1302.10,Bank2,Visa_Debit,7494 6468 8126 2565,New York City,Rochester,magnetic,1,f,...,0.523820,-2.649406,1.531708,-4.176390,3.584615,-1.023954,-0.502471,-1.891966,0.878417,-1.541942
9,21,1861.30,Bank6,Visa_Debit,1616 8157 5881 1644,West Hartford,New Haven,internet,1,m,...,-1.000611,-4.458008,3.346850,-5.534491,6.835802,-0.299803,0.095951,-2.440419,1.286301,-2.766438


In [19]:
ta[colsout]

,Age,Amount,bank,cardType,entry_type,gender,transaction,v1,v2,v3,v4,fraud
0,45,170.60,Bank8,Visa_Debit,chip,f,ATM,-1.550273,1.088689,-2.393388,1.008733,1
1,70,4441.70,Bank12,Visa,internet,f,INTERNET,-2.335655,2.225380,-3.379450,2.178538,1
2,23,10.00,Bank0,Visa_Debit,chip,f,ATM,-3.843009,3.375110,-5.492893,6.136378,1
3,32,11.80,Bank6,Visa_Debit,magnetic,m,ATM,1.296231,0.417447,0.193963,0.901644,1
4,74,37.90,Bank2,Master,internet,m,INTERNET,-3.600544,4.519047,-6.340884,6.214767,1
5,21,224.70,Bank0,Visa,chip,m,ATM,-4.666500,-3.952320,0.206094,5.153525,1
6,25,15.20,Bank1,Visa_Debit,internet,m,INTERNET,1.081234,0.416414,0.862919,2.520863,1
7,31,180.00,Bank6,Visa,internet,m,INTERNET,-1.108478,3.448953,-6.216972,3.021052,1
8,78,1302.10,Bank2,Visa_Debit,magnetic,f,ATM,0.523820,1.531708,-4.176390,3.584615,1
9,21,1861.30,Bank6,Visa_Debit,internet,m,INTERNET,-1.000611,3.346850,-5.534491,6.835802,1


In [20]:
ta[colsout].describe()

,Age,Amount,v1,v2,v3,v4,fraud
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,48.906600,195.267700,-0.519241,0.371169,-0.725877,0.458313,0.100000
std,17.925424,871.911878,3.300753,2.414790,3.514918,2.150533,0.300015
min,18.000000,0.000000,-30.821436,-35.616754,-31.103685,-4.872671,0.000000
25%,33.000000,6.670000,-1.110186,-0.536876,-1.264011,-0.764025,0.000000
50%,49.000000,22.995000,-0.174318,0.172644,0.018809,0.147058,0.000000
75%,65.000000,89.235000,1.257655,0.993551,0.959805,1.086963,0.000000
max,79.000000,21258.700000,2.408683,22.057729,3.771745,11.927512,1.000000
